## Category-4-Unsupervised Machine Learning on Unstructured data
Dataset consists of a set of automobile reviews written by Amazon customers. 

** Amazon reviews across categories can be found at: http://jmcauley.ucsd.edu/data/amazon/ **

The objective of the learning program is to extract information from text data using Natural Language Processing (NLP) techniques. This is an illustration of Word2Vec algorithm used in a unsupervised context.

> Good reference is the Kaggle Tutorial titled: "Bag of Words Meets Bags of Popcorn" and available at this link https://www.kaggle.com/c/word2vec-nlp-tutorial

In [1]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# NLP libraries
import re, nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
#nltk.download('stopwords')

# Word2Vec packages
import gensim
from gensim.models import Word2Vec

C:\Users\Karthikeyan\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
#    level=logging.INFO)

In [3]:
# load the reviews into a pandas dataframe
data_file_name = "./0.datasets/Automotive_5.json"
df = pd.read_json(data_file_name, lines=True)
print("Dataset contains {0:,} reviews".format(df.shape[0]))
df.head()

Dataset contains 20,473 reviews


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B00002243X,"[4, 4]",5,I needed a set of jumper cables for my new car...,"08 17, 2011",A3F73SC1LY51OO,Alan Montgomery,Work Well - Should Have Bought Longer Ones,1313539200
1,B00002243X,"[1, 1]",4,"These long cables work fine for my truck, but ...","09 4, 2011",A20S66SKYXULG2,alphonse,Okay long cables,1315094400
2,B00002243X,"[0, 0]",5,Can't comment much on these since they have no...,"07 25, 2013",A2I8LFSN2IS5EO,Chris,Looks and feels heavy Duty,1374710400
3,B00002243X,"[19, 19]",5,I absolutley love Amazon!!! For the price of ...,"12 21, 2010",A3GT2EWQSO45ZG,DeusEx,Excellent choice for Jumper Cables!!!,1292889600
4,B00002243X,"[0, 0]",5,I purchased the 12' feet long cable set and th...,"07 4, 2012",A3ESWJPAVRPWB4,E. Hernandez,"Excellent, High Quality Starter Cables",1341360000


** We are interested in processing the "reviewText" column **

In [4]:
# Get the ReviewText into a new dataframe
all_reviews = pd.DataFrame(df['reviewText'])
all_reviews.shape

(20473, 1)

In [5]:
# Select sample set of reviews as appropriate
sample_reviews = all_reviews.head(1000)

** Text Preprocessing Starts **

In [6]:
# Function to preprocess text - Remove non-letters, lowercase, optionally remove stop words
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    # 1. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
    #
    # 2. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 3. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 4. Return a list of words
    return(words)

In [7]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')  

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence,remove_stopwords=True ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Karthikeyan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Get the sentences from the review corpus. Sentences are the inputs to Word2Vec algorithm
sentences = []  # Initialize an empty list of sentences

for review in sample_reviews['reviewText']:
    sentences += review_to_sentences(review, tokenizer)
print("We have {0:,} sentences".format(len(sentences)))

We have 4,813 sentences


** Text Preprocessing completed, sentences created and Word2Vec starts **

In [9]:
## Setting the parameters for Word2Vec

import multiprocessing

# Set values for various parameters for Word2Vec
num_features = 50    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = multiprocessing.cpu_count()      # Number of threads to run in parallel
context_size = 3          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


In [10]:
# Initialize Word2Vec. Since sentences are supplied it will build vocab and then train the model
model_name = "auto2vec_model"
if 1==1:  # Set this to 0==1 if you want to utilize the already trained model
    auto2vec_trained = Word2Vec(sentences, workers=num_workers,sg=1, \
                size=num_features, min_count = min_word_count, \
                window = context_size, sample = downsampling)


    print("The vocabulary is built")
    print("Auto2Vec vocabulary length: ", len(auto2vec_trained.vocab))

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    auto2vec_trained.init_sims(replace=True)

    # It can be helpful to create a meaningful model name and 
    # save the model for later use. You can load it later using Word2Vec.load()    
    auto2vec_trained.save(model_name)
    print ("Training model completed...")

The vocabulary is built
Auto2Vec vocabulary length:  1553
Training model completed...


In [11]:
# load the finished model from disk
print("Loading model from disk")
auto2vec = Word2Vec.load(model_name)
auto2vec.init_sims()

Loading model from disk


In [12]:
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in auto2vec.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda x:(x[2]), reverse=True)

ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

In [13]:
word_vectors = pd.DataFrame(auto2vec.wv.syn0norm[term_indices,:], index=ordered_terms)
#word_vectors[25:75]

** Perform word associations & word algebra **

In [14]:
#auto2vec.most_similar("auto")

In [15]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in auto2vec.most_similar(positive=[token], topn=topn):

        print (u'{:20} {}'.format(word, round(similarity, 3)))

In [16]:
get_related_terms(u'jumper',topn=10)

away                 0.999
reach                0.999
premium              0.999
based                0.999
drop                 0.999
transmission         0.999
believe              0.999
always               0.999
twice                0.999
solution             0.999


In [17]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = auto2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print (term)

In [18]:
word_algebra(add=[u'vehicle', u'lawn'],topn=5)
#word_algebra(add=[u'vehicle', u'lawn'], subtract=[u'wheels'])

hours
cost
removed
single
might


In [19]:
auto2vec.doesnt_match("car truck engine use".split())

'car'

## spaCy &mdash; Industrial-Strength NLP in Python

![spaCy](https://s3.amazonaws.com/skipgram-images/spaCy.png)

spaCy is an industrial-strength natural language processing (NLP) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.
spaCy handles many tasks commonly associated with building an end-to-end natural language processing pipeline:
1. Tokenization
2. Text normalization, such as lowercasing, stemming/lemmatization
3. Part-of-speech tagging
4. Syntactic dependency parsing
5. Sentence boundary detection
6. Named entity recognition and annotation

In the "batteries included" Python tradition, spaCy contains built-in data and models which you can use out-of-the-box for processing general-purpose English language text:
1. Large English vocabulary, including stopword lists
2. Token "probabilities"
3. Word vectors

spaCy is written in optimized Cython, which means it's fast. According to a few independent sources, it's the fastest syntactic parser available in any language. Key pieces of the spaCy parsing pipeline are written in pure C, enabling efficient multithreading (i.e., spaCy can release the GIL).

In [20]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

In [21]:
# Convert all the review text into a long string and print its length
raw_corpus = u"".join(sample_reviews['reviewText']+" ")
print("Raw Corpus contains {0:,} characters".format(len(raw_corpus)))

Raw Corpus contains 441,496 characters


In [22]:
if 1==1: # Make this 0==1, if you don't want to run the analysis again
    parsed_review = nlp(raw_corpus)

In [32]:
if 1==1: # Make this 0==1, if you don't want to run the analysis again
    for num, sentence in enumerate(parsed_review.sents):
        print ('Sentence {}:'.format(num + 1))
        print (sentence)
        print ('')

Sentence 1:
I needed a set of jumper cables for my new car and these had good reviews and were at a good price.  

Sentence 2:
They have been used a few times already and do what they are supposed to - no complaints there.

Sentence 3:
What I will say is that 12 feet really isn't an ideal length.  

Sentence 4:
Sure, if you pull up front bumper to front bumper they are plenty long, but a lot of times you will be beside another car or can't get really close.  

Sentence 5:
Because of this, I would recommend something a little longer than 12'.Great brand - get 16' version though.

Sentence 6:
These long cables work fine for my truck, but the quality seems a little on the shabby side.

Sentence 7:
For the money I was not expecting 200 dollar snap-on jumper cables but these seem more like what you would see at a chinese knock off shop like harbor freight for 30 bucks.

Sentence 8:
Can't comment much on these since they have not yet been used (I will come back and update my review is I find

I keep them in one of my containers on my delivery vehicle.

Sentence 69:
They are high quality.

Sentence 70:
They are 4 gauge cable, so they are perfect and long enough to Start are Utilimaster Diesel trucks when are batteries are run Down.

Sentence 71:
There are more expensive cables and even more sever duty use cables.

Sentence 72:
these are perfect for this type of vehicle in an Emergency.

Sentence 73:
The cost is reasonable as well.

Sentence 74:
With 25 feet, you don't have to get real close to the other vehicle.

Sentence 75:
you can pull up to the back and or side and just run the cables under or down the side of the vehicle. bought these for my k2500

Sentence 76:
suburban plenty of length front to rear.

Sentence 77:
i ended up using them right out of the box as i for a dead battery in my truck.

Sentence 78:
they worked great.

Sentence 79:
several people have complained about the clamps.

Sentence 80:
they worked well for me and i had no issues with poor connections.

S

But about as flexible as I expected.  

Sentence 144:
True welding cable in 2ga is about $3 per foot and per cable.  

Sentence 145:
That's why the best jumper cables cost $200+.  

Sentence 146:
Still I was hoping for a little thinner stranding to promote flexibilty.--

Sentence 147:
Cable to Clamp - Like other reviews have mentioned the cable doesn't go all the way to a copper contact connected to the jaws.  

Sentence 148:
Instead.  

Sentence 149:
It's screwed to the body of the clamp then the copper jaws are also screwed to the body.  

Sentence 150:
Still fine I'm sure.

Sentence 151:
So why buy these cables aren't they overkill?  

Sentence 152:
I've had this argument with people before: "My 6ga cheapies start my truck fine."  

Sentence 153:
Well I will attempt to addess this.  

Sentence 154:
As the number in terms of gauge decreases the ammount of conductor increases.  

Sentence 155:
In other words 2ga has more (perhaps 50-100% more) conductor than 4ga.  

Sentence 156:
This

Sentence 237:
There are end loops to use to tie a load and secure it to the bar.

Sentence 238:
The center post also works well as an attachment point for lumber and pipe.

Sentence 239:
I saw similar racks selling for more than twice as much that provided often less strength and no more versatility than this one.

Sentence 240:
Usually after I have tried something like this I see things that I would improve in the design but not so with this bed extender.

Sentence 241:
This is a must for anybody who wants to keep their car clean between washes.

Sentence 242:
My car sits outside, in parking garages, and old sheds.

Sentence 243:
All of them seem to dump tons of dust on my poor car.

Sentence 244:
This model comes with the wood handle.

Sentence 245:
Others out there have a cheap plastic one.

Sentence 246:
Read the directions on the cover.

Sentence 247:
You're suppose to let it air out after shipping.

Sentence 248:
It has something to do with the wax on the brush.

Sentence 249:
It

This covers a large area, so the time to clean is reduced.3.

Sentence 320:
This is easy to clean (just shake it out) and works great no matter how dirty it gets.4.

Sentence 321:
This is well made and durable.

Sentence 322:
Note: This is not a substitute for going to a car wash or using other cleaners.

Sentence 323:
It does nothing, for example, to remove water spots or bird droppings.

Sentence 324:
It also does nothing to give your car the shine that only wax can do.

Sentence 325:
But it is a great way to maintain your car in between car washes.

Sentence 326:
The first worked so well I decided to purchase a second to keep in the car.  

Sentence 327:
Not why, but as advertised the dirtier it gets the better seems to work.  

Sentence 328:
As long as you use as suggested, dust your car, and not try to clean heavy dirt with it, it will leave your car spotless.

Sentence 329:
Amazingly wipes off dust keeping your car cleaner between washes.

Sentence 330:
If you have a black car, y

...

It was easy to apply and wipe off (I did it by hand).

Sentence 4274:
It took me about 90 minutes to do the All in One and the Sealant (mostly because you have to wait 30 minutes for the sealant to fully dry).

Sentence 4275:
However, make sure you use very little.

Sentence 4276:
Be sure have plenty of light (not sun light) when applying.

Sentence 4277:
I only applied one coat of the AIO and one coat of the KGS.  

Sentence 4278:
I want to see how long it last before I need to reapply to see its longevity.

Sentence 4279:
I was a touch skeptical.

Sentence 4280:
But I can afford to throw it out if I don't like it.  

Sentence 4281:
Well I can say, I will NOT be throwing this out!  

Sentence 4282:
I was so impressed with how my cars looked when I was done that I had to ask my self.

Sentence 4283:
Why didn't I buy this sooner.

Sentence 4284:
And it is true.

Sentence 4285:
A little goes a long way.

Sentence 4286:
This product over-delivers.  

Sentence 4287:
Clean your car very wel

The stuff just works.

Sentence 4591:
If you're one of those folk who likes to keep your car looking clean and shiny whatever the weather, and no matter how old your car (or you) might be, this is stuff you want to have on hand.

Sentence 4592:
It is  gentle, effective and provides the best shine possible, whether your car is nearly new or has 100,000 miles on it.

Sentence 4593:
My daily driver is an original owner, 18 year old, bright red M3, and it still turns heads because it still looks great - thanks in part to regular use of 3M hand glaze.  

Sentence 4594:
Here's what you do - wash your car well, using one of the better car wash solutions like Meguiar's hi-tech wash or P21S car shampoo.  

Sentence 4595:
Dry.  

Sentence 4596:
Then use Griot's Garage (or similar) paint cleaning clay to remove surface contaminants and start to work on the swirls and other imperfections in your paint.  

Sentence 4597:
(You can skip this step, but I've recently discovered this cleaning clay and i

I finally got fed up enough and did some research on-line and noticed the LINCOLN model had the best ratings.

Sentence 4676:
I love the way it locks onto a fitting and doesn't leak all round the sides of the fitting instead of going into the fitting as my old gun did....(made a mess and wasted grease) The Lincoln forces all the grease into the fitting easily...once you snap onto the fitting you only need ONE hand to operate!

Sentence 4677:
This is a quality piece of equipment and worth every penny especially with the best price and free shipping from Amazon.

Sentence 4678:
Now I wont get aggravated and talk myself out of greasing the cars and  lawn tractor as I should be.....

Sentence 4679:
THANK YOU LINCOLN INDUSTRIAL 1134..... Delivered in timely manner and packaged properly.

Sentence 4680:
Easy to use, easy to load as well.

Sentence 4681:
I have been using it a while now and it is holding up.

Sentence 4682:
Great transaction.

Sentence 4683:
My preference was to purchase a gr

Good and tight fit with a quality feeling rubber shielding the metal.

Sentence 4963:
The wire seems pretty thick.

Sentence 4964:
I haven't spliced it so i'm not sure if its a copper or aluminum wire.

Sentence 4965:
I guess they could have made it of poorer quality and it would still work but for this one they decided to take an extra step and should last a long time without issues.

Sentence 4966:
Fast service, great pricing and to top it off I liked this so well I will be back and will recommend to others.

Sentence 4967:
I have no reservation about this product or seller.

Sentence 4968:
I have some fog lights that I use on the back of my truck to light up the sides of my 5th wheel when I am pulling in to a camping spot at night.

Sentence 4969:
I got these so I can easly connect and disconnect these lights.

Sentence 4970:
They make it very easy to connect/disconnect anything you have that just needs power and ground.

Sentence 4971:
I needed to extend the length of the reach of 

If you need an adapter and don't want to pay double at the store order here and get great shipping and receiving of the product.

Sentence 5039:
Works great with all my vehicles and trailers.

Sentence 5040:
Thank you again Amazon for the selling another good product cheaper...

Sentence 5041:
Bought this for my 2012 Tahoe, works really good, makes good contact and no issues so far.  

Sentence 5042:
Real good purchase.

Sentence 5043:
I like these allot they do a great job . Screwed up and lost one so I replaced it .

Sentence 5044:
I love the test lights on this one and they have a 7 5 4 that is very convenient .

Sentence 5045:
I prefer to wire evey truck and car with the 7 blades and go from there with adapters .

Sentence 5046:
As long as your vehicle was wired right to USA uniform standards you will be pleased .

Sentence 5047:
They are a little bulky but work very well .

Sentence 5048:
i like how short this terminal connector is, much shorter then other 7-way plugs i have seen.

In [24]:
if 0==1: # Make this 0==1, if you don't want to run the analysis again
    for num, entity in enumerate(parsed_review.ents):
        print ('Entity {}:'.format(num + 1), entity, '-', entity.label_)
        print ('')

In [25]:
if 1==1: # Make this 0==1, if you don't want to run the analysis again
    token_text = [token.orth_ for token in parsed_review]
    token_pos = [token.pos_ for token in parsed_review]

    pos_dataframe = pd.DataFrame(list(zip(token_text, token_pos)),columns=['token_text', 'part_of_speech'])


In [30]:
pos_dataframe

,token_text,part_of_speech
0,I,PRON
1,needed,VERB
2,a,DET
3,set,NOUN
4,of,ADP
5,jumper,NOUN
6,cables,NOUN
7,for,ADP
8,my,ADJ
9,new,ADJ


In [34]:
if 1==1: # Make this 0==1, if you don't want to run the analysis again
    token_lemma = [token.lemma_ for token in parsed_review]
    token_shape = [token.shape_ for token in parsed_review]

    print(pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
                 columns=['token_text', 'token_lemma', 'token_shape']))

      token_text token_lemma token_shape
0              I      -PRON-           X
1         needed        need        xxxx
2              a           a           x
3            set         set         xxx
4             of          of          xx
5         jumper      jumper        xxxx
6         cables       cable        xxxx
7            for         for         xxx
8             my      -PRON-          xx
9            new         new         xxx
10           car         car         xxx
11           and         and         xxx
12         these       these        xxxx
13           had        have         xxx
14          good        good        xxxx
15       reviews      review        xxxx
16           and         and         xxx
17          were          be        xxxx
18            at          at          xx
19             a           a           x
20          good        good        xxxx
21         price       price        xxxx
22             .           .           .
23              

In [33]:
if 1==1: # Make this 0==1, if you don't want to run the analysis again
    token_entity_type = [token.ent_type_ for token in parsed_review]
    token_entity_iob = [token.ent_iob_ for token in parsed_review]

    print(pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
                 columns=['token_text', 'entity_type', 'inside_outside_begin']))

      token_text entity_type inside_outside_begin
0              I                                O
1         needed                                O
2              a                                O
3            set                                O
4             of                                O
5         jumper                                O
6         cables                                O
7            for                                O
8             my                                O
9            new                                O
10           car                                O
11           and                                O
12         these                                O
13           had                                O
14          good                                O
15       reviews                                O
16           and                                O
17          were                                O
18            at                                O


In [28]:
if 1==1: # Make this 0==1, if you don't want to run the analysis again
    token_attributes = [(token.orth_,
                         token.prob,
                         token.is_stop,
                         token.is_punct,
                         token.is_space,
                         token.like_num,
                         token.is_oov)
                        for token in parsed_review]

    df = pd.DataFrame(token_attributes,
                      columns=['text',
                               'log_probability',
                               'stop?',
                               'punctuation?',
                               'whitespace?',
                               'number?',
                               'out of vocab.?'])

    df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                           .applymap(lambda x: u'Yes' if x else u''))

    df

In [29]:
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,I,-4.064181,Yes,,,,
1,needed,-9.102435,,,,,
2,a,-3.983075,Yes,,,,
3,set,-8.572532,,,,,
4,of,-4.128464,Yes,,,,
5,jumper,-19.579313,,,,,
6,cables,-19.579313,,,,,
7,for,-4.913970,Yes,,,,
8,my,-5.918125,Yes,,,,
9,new,-7.566638,,,,,
